# MCP CLIENT SETUP


## 客户端架构
- api key，从 llm 平台获取
- 核心架构primitives：
    1. sampling: 从llm获取回复
    2. roots: 客户端制定服务器应重点关注的目录，协调机制
    3. elicitation：询问用户信息，征得用户同意


### python 环境设置
创建新的python项目
```bash
uv init mcp-client
cd mcp-client
```
创建新的虚拟环境
```bash
uv venv
source .venv/bin/activate  # macOS/Linux
.venv\Scripts\activate     # Windows
```
安装包裹
```bash
uv add mcp anthropic python-dotenv
```
Remove the boilerplate files:
```bash
rm main.py
touch client.py
```
设置llm api key
```bash
echo "ANTHROPIC_API_KEY=your-api-key-here" > .env
echo ".env" >> .gitignore
```

Node.js 环境设置建新的node项目
```bash
mkdir mcp-client-typescript
cd mcp-client-typescript
npm init -y
```
设置依赖
```bash
npm install @anthropic-ai/sdk @modelcontextprotocol/sdk dotenv
npm install -D @types/node typescript
```
Create the index.ts file:
```bash
touch index.ts
```
更新json 格式
```json
{
  "type": "module",
  "scripts": {
    "build": "tsc && chmod 755 build/index.js"
  }
}
```
设置环境文件根据API key
```bash
echo "ANTHROPIC_API_KEY=your-api-key-here" > .env
```


## 基础客户端架构
最基本的class设置
```python
import asyncio
from typing import Optional
from contextlib import AsyncExitStack

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from 使用的llm模型 import 使用的llm模型
from dotenv import load_dotenv

load_dotenv()  # load environment variables from .env

class MCPClient:
    def __init__(self):
        # Initialize session and client objects
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
        self.使用的llm模型 = 使用的llm模型()
    # methods will go here
```
在这之后加入methods ：
- `__aenter__` 和 `__aexit__`：用于异步上下文管理，确保会话在使用后被正确关闭。
- `connect`：连接到 MCP 服务器，初始化会话。
- `disconnect`：关闭会话，释放资源。
- `call`：发送 JSON-RPC 请求到服务器，获取响应。

**这之后全部跟在def 的下面一层**


### 连接MCP服务器
```python
async def connect_to_server(self, server_script_path: str):
    """Connect to an MCP server

    Args:
        server_script_path: Path to the server script (.py or .js)
    """
    is_python = server_script_path.endswith('.py')
    is_js = server_script_path.endswith('.js')
    if not (is_python or is_js):
        raise ValueError("Server script must be a .py or .js file")

    command = "python" if is_python else "node"
    server_params = StdioServerParameters(
        command=command,
        args=[server_script_path],
        env=None
    )

    stdio_transport = await self.exit_stack.enter_async_context(stdio_client(server_params))
    self.stdio, self.write = stdio_transport
    self.session = await self.exit_stack.enter_async_context(ClientSession(self.stdio, self.write))

    await self.session.initialize()

    # List available tools
    response = await self.session.list_tools()
    tools = response.tools
    print("\nConnected to server with tools:", [tool.name for tool in tools])

    # Store available tools for later use
    self.tools = {tool.name: tool for tool in tools}
    print("Available tools:", self.tools)
    return self.tools
    # methods will go here
    '''



### 用户问询逻辑
这是客户端的核心部分，处理客户输入以及处理工具调用
```python
async def process_query(self, query: str) -> str:
    """Process a query using Claude and available tools"""
    messages = [
        {
            "role": "user",
            "content": query
        }
    ]

    response = await self.session.list_tools()
    available_tools = [{
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    } for tool in response.tools]

    # Initial Claude API call
    response = self.anthropic.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        messages=messages,
        tools=available_tools
    )

    # Process response and handle tool calls
    final_text = []

    assistant_message_content = []
    for content in response.content:
        if content.type == 'text':
            final_text.append(content.text)
            assistant_message_content.append(content)
        elif content.type == 'tool_use':
            tool_name = content.name
            tool_args = content.input

            # Execute tool call
            result = await self.session.call_tool(tool_name, tool_args)
            final_text.append(f"[Calling tool {tool_name} with args {tool_args}]")

            assistant_message_content.append(content)
            messages.append({
                "role": "assistant",
                "content": assistant_message_content
            })
            messages.append({
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": content.id,
                        "content": result.content
                    }
                ]
            })

            # Get next response from Claude
            response = self.anthropic.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=1000,
                messages=messages,
                tools=available_tools
            )

            final_text.append(response.content[0].text)

    return "\n".join(final_text)
    '''


### 交互式界面
建立对话轮回，清理功能
```python
async def chat_loop(self):
    """Run an interactive chat loop"""
    print("\nMCP Client Started!")
    print("Type your queries or 'quit' to exit.")

    while True:
        try:
            query = input("\nQuery: ").strip()

            if query.lower() == 'quit':
                break

            response = await self.process_query(query)
            print("\n" + response)

        except Exception as e:
            print(f"\nError: {str(e)}")

async def cleanup(self):
    """Clean up resources"""
    await self.exit_stack.aclose()
```


### 接入点，主要执行逻辑
```python
async def main():
    if len(sys.argv) < 2:
        print("Usage: python client.py <path_to_server_script>")
        sys.exit(1)

    client = MCPClient()
    try:
        await client.connect_to_server(sys.argv[1])
        await client.chat_loop()
    finally:
        await client.cleanup()

if __name__ == "__main__":
    import sys
    asyncio.run(main())
```


## 完善思路
1. 加强工具调用处理，在‘process_query’中增加工具调用处理,错误处理
2. 回复处理，设置回复格式
3. 用户界面

